***BIG DATA RESEARCH PROJECT: INVESTIGATING DONATION BIAS ON PHYSICIAN DRUG PRESCRIPTIONS***

---




***STEP 1: PARSING/CLEANING THE DATA ***

1.   [Accepted donations made to doctors from 2013 to 2017 ](https://openpaymentsdata.cms.gov/browse) 
2.   [Prescriptions made by Medicare physicians](https://data.cms.gov/Medicare-Part-D/Medicare-Provider-Utilization-and-Payment-Data-201/3z4d-vmhm)
**NOTE: try to find more calendar years** [Link to more years of data](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Part-D-Prescriber.html)
3. [List of drugs, their producer, and their prices](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Information-on-Prescription-Drugs/Downloads/Medicare_PartD_Drug_Spending_Data.zip)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly
import dask.dataframe as dd

In [2]:
medicare_drug_spending_man = pd.read_csv('spending_manufacturer_summary_2016.csv')
medicare_drug_spending_man


,Brand Name,Generic Name,Manufacturer,Total Spending,Total Dosage Units,Total Claims,Total Beneficiaries,Average Spending Per Dosage Unit (Weighted),Average Spending Per Claim,Average Spending Per Beneficiary
0,1st Tier Unifine Pentips,"Pen Needle, Diabetic",Owen Mumford Us,"$215,930.43","1,050,323","8,897","3,423",$0.21,$24.27,$63.08
1,1st Tier Unifine Pentips Plus,"Pen Needle, Diabetic",Owen Mumford Us,"$378,001.19","1,750,607","14,611","6,456",$0.22,$25.87,$58.55
2,Abacavir,Abacavir Sulfate,Apotex Corp,"$2,491,345.83","533,661","8,449","2,320",$4.67,$294.87,"$1,073.86"
3,Abacavir,Abacavir Sulfate,Aurobindo Pharm,"$7,154,093.03","1,558,949","25,385","4,602",$4.59,$281.82,"$1,554.56"
4,Abacavir,Abacavir Sulfate,Camber Pharmace,"$6,766,490.99","1,547,860","23,189","4,580",$4.37,$291.80,"$1,477.40"
5,Abacavir,Abacavir Sulfate,Mylan,"$2,925,844.14","613,193","9,503","2,855",$4.77,$307.89,"$1,024.81"
6,Abacavir-Lamivudine,Abacavir Sulfate/Lamivudine,Prasco Labs,"$6,765,112.20","199,418","6,430",,$33.92,"$1,052.12",
7,Abacavir-Lamivudine,Abacavir Sulfate/Lamivudine,Teva USA,"$7,990,669.89","240,422","7,476",,$33.24,"$1,068.84",
8,Abacavir-Lamivudine-Zidovudine,Abacavir/Lamivudine/Zidovudine,Lupin Pharmaceu,"$12,787,721.57","608,455","9,768","1,333",$21.02,"$1,309.14","$9,593.19"
9,Abelcet,Amphotericin B Lipid Complex,Sigma-Tau/Leadi,"$752,938.84","85,650",476,,$8.79,"$1,581.80",


In [3]:
file = "/Volumes/Samsung_T5/Misc/out_open_payments.csv"
open_payments = dd.read_csv(file,
                            dtype={'Associated_Drug_or_Biological_NDC_4': 'object',
       'Covered_or_Noncovered_Indicator_4': 'object',
       'Covered_or_Noncovered_Indicator_5': 'object',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4': 'object',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5': 'object',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4': 'object',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5': 'object',
       'Product_Category_or_Therapeutic_Area_4': 'object',
       'Product_Category_or_Therapeutic_Area_5': 'object',
       'Recipient_Postal_Code': 'object',
       'Recipient_Province': 'object'},
                           low_memory=False)

In [4]:
open_payments.head(10)

,Physician_Profile_ID,Name,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Recipient_City,Recipient_State,Recipient_Zip_Code,Recipient_Country,Recipient_Province,...,Associated_Drug_or_Biological_NDC_3,Covered_or_Noncovered_Indicator_4,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4,Product_Category_or_Therapeutic_Area_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Associated_Drug_or_Biological_NDC_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5
0,1261770.0,Donald Delisi,NaN,NaN,NaN,Bend,OR,97702,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,436996.0,Todd Schock,NaN,NaN,NaN,Bend,OR,97702,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,640211.0,Keith Krueger,NaN,NaN,NaN,Bend,OR,97702,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,389063.0,Arthur Pancioli,NaN,NaN,NaN,Cincinnati,OH,45243,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,389063.0,Arthur Pancioli,NaN,NaN,NaN,Cincinnati,OH,45243,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,389063.0,Arthur Pancioli,NaN,NaN,NaN,Cincinnati,OH,45243,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,74635.0,JOHN FORDTRAN,NaN,NaN,NaN,DALLAS,TX,75246-2017,United States,NaN,...,10572-302-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,74635.0,JOHN FORDTRAN,NaN,NaN,NaN,DALLAS,TX,75246-2017,United States,NaN,...,10572-302-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,74635.0,JOHN FORDTRAN,NaN,NaN,NaN,DALLAS,TX,75246-2017,United States,NaN,...,10572-302-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,74635.0,JOHN FORDTRAN,NaN,NaN,NaN,DALLAS,TX,75246-2017,United States,NaN,...,10572-302-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
open_payments.columns

Index(['Physician_Profile_ID', 'Name', 'Teaching_Hospital_CCN',
       'Teaching_Hospital_ID', 'Teaching_Hospital_Name', 'Recipient_City',
       'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country',
       'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type',
       'Physician_Specialty',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
       'Total_Amount_of_Payment_USDollars', 'Date_of_Payment',
       'Number_of_Payments_Included_in_Total_Amount',
       'Form_of_Payment_or_Transfer_of_Value',
       'Nature_of_Payment_or_Transfer_of_Value',
       'Physician_Ownership_Indicator',
       'Third_Party_Payment_Recipient_Indicator',
       'Name_of_Third

In [6]:
to_remove = ['Covered_or_Noncovered_Indicator_2',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2',
       'Product_Category_or_Therapeutic_Area_2',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2',
       'Associated_Drug_or_Biological_NDC_2',
       'Covered_or_Noncovered_Indicator_3',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3',
       'Product_Category_or_Therapeutic_Area_3',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
       'Associated_Drug_or_Biological_NDC_3',
       'Covered_or_Noncovered_Indicator_4',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4',
       'Product_Category_or_Therapeutic_Area_4',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4',
       'Associated_Drug_or_Biological_NDC_4',
       'Covered_or_Noncovered_Indicator_5',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5',
       'Product_Category_or_Therapeutic_Area_5',
       'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5']

df = open_payments.drop(to_remove, axis =1 )

In [7]:
df.columns

Index(['Physician_Profile_ID', 'Name', 'Teaching_Hospital_CCN',
       'Teaching_Hospital_ID', 'Teaching_Hospital_Name', 'Recipient_City',
       'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country',
       'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type',
       'Physician_Specialty',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
       'Total_Amount_of_Payment_USDollars', 'Date_of_Payment',
       'Number_of_Payments_Included_in_Total_Amount',
       'Form_of_Payment_or_Transfer_of_Value',
       'Nature_of_Payment_or_Transfer_of_Value',
       'Physician_Ownership_Indicator',
       'Third_Party_Payment_Recipient_Indicator',
       'Name_of_Third

In [8]:
r = ['Physician_Ownership_Indicator',
       'Third_Party_Payment_Recipient_Indicator',
       'Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value',
       'Charity_Indicator', 'Contextual_Information',
       'Dispute_Status_for_Publication', 'Covered_or_Noncovered_Indicator_1']
df = df.drop(r, axis=1)

In [9]:
df.columns

Index(['Physician_Profile_ID', 'Name', 'Teaching_Hospital_CCN',
       'Teaching_Hospital_ID', 'Teaching_Hospital_Name', 'Recipient_City',
       'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country',
       'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type',
       'Physician_Specialty',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
       'Total_Amount_of_Payment_USDollars', 'Date_of_Payment',
       'Number_of_Payments_Included_in_Total_Amount',
       'Form_of_Payment_or_Transfer_of_Value',
       'Nature_of_Payment_or_Transfer_of_Value',
       'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1',
       'Product_Category_or_Therapeutic_Are

In [10]:
import dask_ml.cluster

In [ ]:
km = dask_ml.cluster.KMeans(n_clusters=3, init_max_iter=2, oversampling_factor=10)
km.fit(df)

In [ ]:
open_payments['Physician_Specialty'].head(1000)

In [ ]:
sns.distplot(open_payments['Total_Amount_of_Payment_USDollars'])

In [ ]:
payments = open_payments.compute()

In [ ]:
# r = 'Recipient_City'
# t = 'Total_Amount_of_Payment_USDollars'


# sns.catplot(x=r, y=t, data=open_payments)

In [ ]:
# # x = 'Recipient_City'
# y = 'Total_Amount_of_Payment_USDollars'


# plt.hist(payments[y])



In [ ]:
# from sklearn.linear_model import LinearRegression

# sns.catplot(x='Physician_Specialty', y='Total_Amount_of_Payment_USDollars', data = payments)

In [ ]:
import statsmodels

In [ ]:
filename = "/Volumes/Samsung_T5/Misc/PartD_Prescriber_PUF_NPI_Drug_16.txt"

prescriptions = dd.read_csv(filename, sep="\t")

In [ ]:
columns_dropped_prescriptions = ["description_flag", "bene_count_ge65", "bene_count_ge65_suppress_flag", 
                                 "ge65_suppress_flag", "total_claim_count_ge65",
                                "total_30_day_fill_count_ge65", "total_day_supply_ge65", "total_drug_cost_ge65"]

prescriptions = prescriptions.drop(columns_dropped_prescriptions, axis=1)

In [ ]:
prescriptions.columns

In [ ]:
prescriptions.head(1000)

In [ ]:
prescriptions['name']  = prescriptions['nppes_provider_last_org_name'].str.title() + " " + prescriptions['nppes_provider_first_name'].str.title()

In [ ]:
to_remove = ['nppes_provider_last_org_name', 'nppes_provider_first_name']

pres = prescriptions.drop(to_remove, axis=1)

In [ ]:
cols =['name'] + [col for col in pres.columns if col != 'name']

pres2 = pres[cols]

In [ ]:
pres2.head(5)

In [ ]:
pres3 = pres2.compute()

In [ ]:
orthopaedic = pres3[pres3['specialty_description'] == 'Orthopaedic Surgery']

In [ ]:
pediatrics = pres3[pres3['specialty_description'].str.startswith('Pediatrics')]

In [ ]:
df = pd.DataFrame(orthopaedic.groupby(["drug_name", "generic_name", "name"])['total_claim_count'].sum())

In [ ]:
df['total_claim_count'].sort_values(ascending=False)

In [ ]:
ortho_names = df.index.get_level_values(2) 

In [ ]:
len(ortho_names)

In [ ]:
import re

In [ ]:
g = []
for x in orthopaedic['generic_name']:
    x = re.sub(r'\/', '-', x)
    g.append(x)

In [ ]:
df.index.names

In [ ]:
orthopaedic.shape

In [ ]:
orthopaedic['generic_name'] = g

In [ ]:
orthopaedic['generic_name'] 

In [ ]:
generic = []
generic_docs = []

brand = []
brand_docs = []

for x, y, z in zip(orthopaedic['generic_name'], orthopaedic['drug_name'], orthopaedic['name']):
    if (x == y):
        generic_docs.append(z)
        generic.append(x)
    else:
        brand_docs.append(z)
        brand.append(y)


In [ ]:
ortho_generic = pd.DataFrame()

ortho_generic['doctors'] = generic_docs
ortho_generic['generic_name'] = generic

In [ ]:
ortho_generic.shape

In [ ]:
ortho_brand = pd.DataFrame()

ortho_brand['doctors'] = brand_docs
ortho_brand['brand_name'] = brand

In [ ]:
ortho_brand

In [ ]:
ortho_brand.to_csv('ortho_brand.csv', index=False)

In [ ]:
# for row in orthopaedic.itertuples(index=True, name='Series'):
#     r = pd.DataFrame(row)
#     brand = row[6]
#     generic=  row[7]
#     if brand == re.sub(r'\/', '-', generic):
#         ortho_generic = ortho_generic.append(r)   
#     else:
#         pd.concat([ortho_brand, r], axis=0)

In [ ]:
len(generic)

In [ ]:
len(brand)

# Networkx

In [ ]:
import networkx as nx

In [ ]:
#list_of_edges = list(pres3['drug_name'])
b = list(pres3['drug_name']) 
a = list(pres3['name'])

In [ ]:
c = []
for i, j in zip(a, b):
    c.append((i, j))
c

In [ ]:
list_of_edges = c

In [ ]:
G = nx.Graph()

G.add_nodes_from(a)
G.add_nodes_from(b)

In [ ]:
G.add_edges_from(c)

In [ ]:
# nx.draw(G)

In [ ]:
prescriptions.columns

# ***STEP 2: VISUALIZATION THROUGH HISTOGRAMS AND NETWORKS***


1.   Visualizing Big Pharma companies' connections to doctors (has been donated to) 
2.   Histogram to show drug price distribution (how many Big Pharma drugs are in the upper price range)

Python network visualization modules: Netwulf, [Graph-Tool](https://graph-tool.skewed.de/), [Plotly](https://plot.ly/python/network-graphs/#new-to-plotly)
 



***STEP 3: MACHINE LEARNING***

1.   How monetary value of donation accepted might predict how much of a company’s drugs the doctor is likely to prescribe in a given year using **RANDOM FOREST CLASSIFIERS** and **GRADIENT-BOOSTED DECISION TREES**
2. Could also investigate specific type of donation: i.e. is free food the easiest way to influence a doctor?




```
# This is formatted as code
```

***STEP 4: MAPREDUCE***

1.   Relational triangles between a physician's medical specialty and acceptance of donations from Big Pharma


***STEP 5: BLOG POST TO SHOW FINDINGS***



---


***MISCELLANEOUS QUESTIONS/CONCERNS:***

1.   List item
2.   List item

